In [1]:
import os
import pandas as pd
from pydub import AudioSegment
import speech_recognition as sr
from speechFileMP3ToText import speechFileMP3ToText
from creating_test_files_subset import creating_test_files_subset

path1 = "C:/Users/Simon/OneDrive/Documents/FISE3/Semestre 9/Spe-Info2-Big Data/Big Data Project/en/clips/"
filtered_test_df = creating_test_files_subset(path1)

for filename in os.listdir('./en_test_full_subset/'):
    if filename != "target_directory":
        output = speechFileMP3ToText(filename)
        filtered_test_df.loc[filtered_test_df['path'] == filename, 'predicted_sentence'] = output
        filtered_test_df = filtered_test_df.reindex(filtered_test_df.loc[filtered_test_df['path'] == filename].index.union(filtered_test_df.index))
# filtered_test_df


# filtered_test_df.size #99

# une nouvelle colonne "correct_predicted" qui est un boolean et qui possède True si les champs "path" et "predicted_sentence" sont égales
#Le problème c'est si on a dans correct_predicted "3" ou "444"
#-->On va utiliser fuzzywuzzy (on a aussi besoin d'installer la python-Levenshtein)
# filtered_test_df['correct_predicted'] = filtered_test_df.apply(lambda row: row['sentence'] == row['predicted_sentence'], axis=1)
from fuzzywuzzy import fuzz
#90 ça ne suffit pas, tester à la main pour voir?
#Même avec 50, ça ne marche pas
# filtered_test_df["correct_predicted"] = filtered_test_df.apply(lambda row: fuzz.token_set_ratio(row['sentence'], row['predicted_sentence']) > 50, axis=1)


#On va utiliser une autre approche
#C'est bcp plus long, ça a pris ~1min
#ça a résolu le problème du "3" mais pas du "444"

def convert_number(n):
    if n == "1":
        return "one"
    elif n == "2":
        return "two"
    elif n == "3":
        return "three"
    elif n == "4":
        return "four"
    else:
        return n

filtered_test_df['predicted_sentence'] = filtered_test_df['predicted_sentence'].apply(convert_number)
filtered_test_df["correct_predicted"] = filtered_test_df.apply(lambda row: row['sentence'] == row['predicted_sentence'], axis=1)

filtered_test_df


,client_id,path,sentence,up_votes,down_votes,age,gender,accent,locale,segment,predicted_sentence,correct_predicted
0,02a86ce07b7b7d4dfb369c632369b6d7ebcff87472beae...,common_voice_en_22005778.mp3,yes,3,0,NaN,NaN,NaN,en,Benchmark,yes,True
1,042bc2389193679673f4860dd38356e5c1bec5bf9b95d6...,common_voice_en_22310890.mp3,yes,3,0,NaN,NaN,NaN,en,Benchmark,yes,True
2,099b4fa0c62e39a22db1e1de8d756cb1eb29f122dfab99...,common_voice_en_21960400.mp3,three,2,1,NaN,NaN,NaN,en,Benchmark,three,True
3,0b5e0bb806dba1eb60c51b25420e40d7daffff30ca29b9...,common_voice_en_22316853.mp3,yes,2,0,NaN,NaN,NaN,en,Benchmark,yes,True
4,0bdbe5114e8f69ee5a4e2b46ed365bd10f7bfa5addbaba...,common_voice_en_22327660.mp3,yes,3,0,NaN,NaN,NaN,en,Benchmark,yes,True
...,...,...,...,...,...,...,...,...,...,...,...,...
3462,c2cf4027a6375de35e9958fbec94e8af6eba58837bdc51...,common_voice_en_21951428.mp3,two,2,0,NaN,NaN,NaN,en,Benchmark,two,True
3463,c2cf4027a6375de35e9958fbec94e8af6eba58837bdc51...,common_voice_en_21951430.mp3,no,4,0,NaN,NaN,NaN,en,Benchmark,no,True
3464,c30d3c26910df3d2538bf03d190eb3b5bcc2a183b793c8...,common_voice_en_22150155.mp3,yes,2,0,NaN,NaN,NaN,en,Benchmark,yes,True
3465,c30d3c26910df3d2538bf03d190eb3b5bcc2a183b793c8...,common_voice_en_22150157.mp3,four,2,1,NaN,NaN,NaN,en,Benchmark,or her,False


In [11]:
percentage_of_true = 100 * filtered_test_df['correct_predicted'].value_counts()[True] / filtered_test_df.shape[0]
print(percentage_of_true)
#69.62792039226997% of True. We can still make an effort to clean the results!

filtered_test_df.to_csv('filtered_test_3467rows.csv', index=False)
# To read the csv file back
# filtered_test_df = pd.read_csv('filtered_test_3467rows.csv')


69.62792039226997


In [8]:
filtered_test_df = pd.read_csv('filtered_test_3467rows.csv')
filtered_test_df

,client_id,path,sentence,up_votes,down_votes,age,gender,accent,locale,segment,predicted_sentence,correct_predicted
0,02a86ce07b7b7d4dfb369c632369b6d7ebcff87472beae...,common_voice_en_22005778.mp3,yes,3,0,NaN,NaN,NaN,en,Benchmark,yes,True
1,042bc2389193679673f4860dd38356e5c1bec5bf9b95d6...,common_voice_en_22310890.mp3,yes,3,0,NaN,NaN,NaN,en,Benchmark,yes,True
2,099b4fa0c62e39a22db1e1de8d756cb1eb29f122dfab99...,common_voice_en_21960400.mp3,three,2,1,NaN,NaN,NaN,en,Benchmark,three,True
3,0b5e0bb806dba1eb60c51b25420e40d7daffff30ca29b9...,common_voice_en_22316853.mp3,yes,2,0,NaN,NaN,NaN,en,Benchmark,yes,True
4,0bdbe5114e8f69ee5a4e2b46ed365bd10f7bfa5addbaba...,common_voice_en_22327660.mp3,yes,3,0,NaN,NaN,NaN,en,Benchmark,yes,True
...,...,...,...,...,...,...,...,...,...,...,...,...
3462,c2cf4027a6375de35e9958fbec94e8af6eba58837bdc51...,common_voice_en_21951428.mp3,two,2,0,NaN,NaN,NaN,en,Benchmark,two,True
3463,c2cf4027a6375de35e9958fbec94e8af6eba58837bdc51...,common_voice_en_21951430.mp3,no,4,0,NaN,NaN,NaN,en,Benchmark,no,True
3464,c30d3c26910df3d2538bf03d190eb3b5bcc2a183b793c8...,common_voice_en_22150155.mp3,yes,2,0,NaN,NaN,NaN,en,Benchmark,yes,True
3465,c30d3c26910df3d2538bf03d190eb3b5bcc2a183b793c8...,common_voice_en_22150157.mp3,four,2,1,NaN,NaN,NaN,en,Benchmark,or her,False


In [4]:
filtered_test_df.to_excel('filtered_test_3467rows.xlsx', index=False)
# To read the excel file back
# df = pd.read_excel('file_name.xlsx')


In [14]:
unique_predicted_sentences = filtered_test_df['predicted_sentence'].unique()
unique_predicted_sentences
#Resultats


# array(['yes', 'three', "<class 'speech_recognition.UnknownValueError'>",
#        'do', '444', 'no', 'one', 'yeah', 'War', 'yes yes yes', 'no no',
#        'play Wonderwall', 'phone', 'yes yys', 'two', 'wine', 'play',
#        'for free', 'four', 'poor', 'far', 'yes yes', 'lol', 'or', 'free',
#        'want', 'nearest', 'white', 'Joe', 'we', '4/4', '222', '22',
#        'fall', 'today', '33', 'solve', 'run', 'tree', 'now', 'door',
#        'when', 'ball', 'lower volume', 'more', 'fur', 'Jess', 'you do',
#        'call', 'volume', 'I know', 'Ford', 'Siri', 'turn the alarm off',
#        'Who', 'Bob', '333', 'no no no', 'play stop', '111', 'kale',
#        'tool', 'chill', 'tune', 'play no', 'nope', '11', '44', 'pause',
#        'wang', 'true', 't', 'honey', 'hi', 'warm', 'boar', 'Wham',
#        'stream', 'Noel', 'what', 'team', 'nice', 'why', 'power', 'nest',
#        'through', 'June', 'home', 'bol bol', 'stop', 'poor poor',
#        'Google', 'forward', 'sleep', 'morning', 'dream', 'cool',
#        'now now', 'yep', 'till', 'Marie', 'please', 'noon', 're', 'foul',
#        'form', 'turtle', 'YouTube', 'Dre', 'low', 'Surrey', 'Green',
#        'Bray', 'pray', 'tomb', '4 or 4', 'saw', 'tube', 'you', 'train',
#        'new', 'ring', 'wind', 'lottery', 'still', 'Poole', 'all', 'mail',
#        'tongue', 'Reed', 'jul', "don't", 'Reiss', 'horror', 'Farm',
#        'fourth', 'Sue', 'telling', 'or her'], dtype=object)

#A la main:
#Les predicted_sentence que l'on peut remplacer:
#444;yeah;yes yes yes;yes yys;...;4/4;4 or 4

#Pattern: 
# 1) Les répétitions!
#Ex: "yes yes yes";
# 2) les variantes
#Ex: "yeah"
#

#Une possibilité, pour les variantes, regarder dans la liste des mots avec leurs fréquences
# 3) Répétitions avec une valeur correcte
#Ex: "yes yys"; 
# 4) ?
#Ex: "fourth"



array(['yes', 'three', "<class 'speech_recognition.UnknownValueError'>",
       'do', '444', 'no', 'one', 'yeah', 'War', 'yes yes yes', 'no no',
       'play Wonderwall', 'phone', 'yes yys', 'two', 'wine', 'play',
       'for free', 'four', 'poor', 'far', 'yes yes', 'lol', 'or', 'free',
       'want', 'nearest', 'white', 'Joe', 'we', '4/4', '222', '22',
       'fall', 'today', '33', 'solve', 'run', 'tree', 'now', 'door',
       'when', 'ball', 'lower volume', 'more', 'fur', 'Jess', 'you do',
       'call', 'volume', 'I know', 'Ford', 'Siri', 'turn the alarm off',
       'Who', 'Bob', '333', 'no no no', 'play stop', '111', 'kale',
       'tool', 'chill', 'tune', 'play no', 'nope', '11', '44', 'pause',
       'wang', 'true', 't', 'honey', 'hi', 'warm', 'boar', 'Wham',
       'stream', 'Noel', 'what', 'team', 'nice', 'why', 'power', 'nest',
       'through', 'June', 'home', 'bol bol', 'stop', 'poor poor',
       'Google', 'forward', 'sleep', 'morning', 'dream', 'cool',
       'now now', '